In [ ]:
from collections import defaultdict as ddict
from typing import DefaultDict, Set
import itertools

import freeman as fm
from redes import constraint

import pandas as pd
import numpy as np
import networkx as nx
import statsmodels.api as sm

In [ ]:
bruno = pd.read_csv(
    "csv/players.csv",
     usecols=("date", "player_name", "team", "player_id", "match_id", "event_id", "rating"),
)

In [ ]:
bruno.loc[:, "date"] = pd.to_datetime(bruno["date"], format="%Y-%m-%d")
bruno.loc[:, "player_name"] = bruno["player_name"].apply(
    lambda x: str(x).encode('ascii', 'backslashreplace').decode('ascii')
)
bruno = bruno[bruno["date"] >= "2017-06-01"]

In [ ]:
bruno.cabeca = bruno.head
print(bruno.shape)
bruno.cabeca()

In [ ]:
upid_rating = bruno.groupby(["player_id", "player_name"])["rating"]
median = upid_rating.median()
median.name = "median"

mean = upid_rating.mean()
mean.name = "mean"

mdiff = median - mean
mdiff.name = "mdiff"

var = upid_rating.var().fillna(upid_rating.last())
var.name = "var"

helionp = pd.concat([median, mean, mdiff, var], axis=1) # x/(N-1)
helionp.ceara = helionp.head
helionp.ceara()

In [ ]:
teams: DefaultDict[str, Set[str]] = ddict(set)
bruno.groupby(["team", "match_id"])["player_id"].apply(
    lambda x: teams[x.name[0]].update(x.values)
);

In [ ]:
with open("csgo.gml", "w+") as f:
    gml = "graph [\n    directed 0\n"
    for (p_id, p_name) in helionp.index:
        gml += "    node [\n"
        gml += f"        id {p_id}\n"
        gml += f"        label \"{p_name}\"\n"
        gml += "    ]\n"
    edges = set()
    for team in teams:
        for edge in itertools.combinations(teams[team], 2):
            edge = tuple(sorted(edge)) # Probably not necessary
            if edge not in edges:
                edges.add(edge)

                gml += "    edge [\n"
                gml += f"        source {edge[0]}\n"
                gml += f"        target {edge[1]}\n"
                gml += "    ]\n"
    gml += "]\n"
    f.write(gml)

In [ ]:
g = fm.load('csgo.gml')

In [ ]:
# g.set_all_nodes(size=10, labpos='hover')
# g.set_all_edges(color=(0, 0, 0, 0.25))
# # g.move("kamada_kawai")
# g.draw()

In [ ]:
con = {}

for n in g.nodes:
    con[n] = constraint(g, n)
helionp.loc[:, "con"] = pd.DataFrame.from_records(list(con.items()))[1].values

In [ ]:
# bc = nx.betweenness_centrality(g)
# helionp.loc[:, "bc"] = pd.DataFrame.from_records(list(bc.items()))[1].values

In [ ]:
helionp.ceara()

In [ ]:
# x = sm.add_constant(helionp[['con']])
x = helionp[['con']]
# x = list(map(lambda x: x[0], helionp.index))
x = [1.1 for _ in range(helionp.shape[0])]
y = [2.2*i for i in range(helionp.shape[0])]
# y = helionp['var']
model = sm.OLS(y, x) # , missing='drop')
result = model.fit()
result.summary()

In [ ]:
# # helionpc = helionp.copy()
# # helionpc = helionpc[['median', 'mean', 'mdiff', 'con']]
# helionp[helionp.isna().any(axis=1)]
# helionp.corr()
#  0.7137
x = helionp["con"]
y = helionp["var"]
import matplotlib.pyplot as plt
plt.hist(x)
plt.title("con")
# plt.plot()
plt.show()
plt.hist(y)
plt.title("var")
# plt.plot()
plt.show()

In [ ]:
result.pvalues

In [ ]:
print(bruno["player_name"].unique().shape)
print(bruno["player_id"].unique().shape)

In [ ]:
s = set()
for i in helionp.index.values:
#     print(i)
    if i[0] in s:
        print(i)
    else:
        s.add(i[0])
#     if helionp.loc[i[0]].shape[0] != 1:
#         print(i)